# Testing Pipelines

### Initiating observation via Phoenix

In [12]:
# %pip pip install arize-phoenix
# %pip install llama-index-callbacks-arize-phoenix

# observability
import phoenix as px
px.launch_app()

import llama_index.core
llama_index.core.set_global_handler("arize_phoenix", endpoint="http://localhost:6006/v1/traces")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Preprocessing pipeline

### Visualizing workflows

In [3]:

import os
from llama_index.utils.workflow import draw_all_possible_flows

from evidence_seeker.preprocessing.preprocessing_separate_listings_workflow import (
    PreprocessingSeparateListingsWorkflow,
)
from evidence_seeker.preprocessing.simple_preprocessing_workflow import SimplePreprocessingWorkflow



# create dir ../tmp if not exists
os.makedirs("../tmp", exist_ok=True)

draw_all_possible_flows(
    PreprocessingSeparateListingsWorkflow, filename="../tmp/PreprocessingSeparateListingsWorkflow.html"
)

draw_all_possible_flows(
    SimplePreprocessingWorkflow, filename="../tmp/SimplePreprocessingWorkflow.html"
)

<class 'NoneType'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.ListAscriptiveClaimsBasedOnAscriptiveAnalysisEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.ListDescriptiveClaimsBasedOnDescriptiveAnalysisEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.AscriptiveAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.DescriptiveAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.NormativeAnalysisEndEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.CollectClarifiedClaimsEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.ListNormativeClaimsBasedOnNormativeAnalysisEvent'>
<class 'evidence_seeker.preprocessing.preprocessing_separate_listings_workflow.NormativeAnalysisEve

### Running Workflows directly

In [1]:

import yaml
from pprint import pprint

from evidence_seeker.backend import get_openai_llm
from evidence_seeker.preprocessing.preprocessing_separate_listings_workflow import (
    PreprocessingSeparateListingsWorkflow,
)
from evidence_seeker.preprocessing.simple_preprocessing_workflow import SimplePreprocessingWorkflow


# config_file = "../config.yaml" 

config_file = "../configs/preprocessing_separate_listings_config.yaml" 
pw = PreprocessingSeparateListingsWorkflow(config_file=config_file)

#config_file = "../configs/simple_preprocessing_config.yaml" 
#pw = SimplePreprocessingWorkflow(config_file=config_file)

examples = [
    "Climate change is the biggest challenge for human kind.",
    "Climate change is getting ever and ever faster.",
    "The rate of climate change is increasing over time.",
]
result = await pw.run(claim=examples[2])
pprint(result)

Loading config from ../configs/simple_preprocessing_config.yaml
Used api key name: kideku_toxicity_app_nim
Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://huggingface.co/api/integrations/dgx/v1).
Analysing ascriptive aspects of claim.
Using workflow model: model_1 for ascriptive_analysis_event
Analysing descriptive aspects of claim.
Using workflow model: model_1 for descriptive_analysis_event
Analysing normative aspects of claim.
Using workflow model: model_1 for normative_analysis_event
Using workflow model: model_1 for list_claims_event
Using workflow model: model_1 for list_claims_event
Number of claims: 1
Negating claim.
Using workflow model: model_1 for negate_claim_event
{'claim': 'The rate of climate change is increasing over time.',
 'clarified_claims': [CheckedClaim(text='The rate of climate change is increasing over time. The factual content of this claim is: Climate change is occurring. There is a rate of climate change. The rate of

### Running ClaimPreprocessor

In [1]:
import yaml
from pprint import pprint

from evidence_seeker.preprocessing import ClaimPreprocessor

config_file = "../configs/preprocessing_separate_listings_config.yaml" 
#config_file = "../configs/simple_preprocessing_config.yaml" 

# Note: Use preprocessor workflow is currently hardset in `ClaimPreprocessor`
preprocessor = ClaimPreprocessor(config_file=config_file)
clarified_claims = await preprocessor(claim="Climate change is the biggest challenge for human kind.") 


Loading config from ../configs/preprocessing_separate_listings_config.yaml
Used api key name: kideku_toxicity_app_nim
Instantiating OpenAILike model (model: meta-llama/Llama-3.1-70B-Instruct,base_url: https://huggingface.co/api/integrations/dgx/v1).
Analysing ascriptive aspects of claim.
Using workflow model: model_1 for ascriptive_analysis_event
Analysing descriptive aspects of claim.
Using workflow model: model_1 for descriptive_analysis_event
Analysing normative aspects of claim.
Using workflow model: model_1 for normative_analysis_event
Using workflow model: model_1 for list_ascriptive_claims_event_based_on_ascriptive_analysis
Using workflow model: model_1 for list_ascriptive_claims_event_based_on_ascriptive_analysis
Using workflow model: model_1 for list_normative_claims_event_based_on_normative_analysis
Using workflow model: model_1 for list_normative_claims_event_based_on_normative_analysis
Using workflow model: model_1 for list_descriptive_claims_event_based_on_descriptive_anal

In [2]:
pprint(clarified_claims)

[CheckedClaim(text='Climate change has significant impacts on human societies', negation='Climate change has no significant impacts on human societies.', uid='f369af94-8095-42cc-b644-6655d061ff45', n_evidence=None, average_confirmation=None, evidential_uncertainty=None, verbalized_confirmation=None, documents=None, confirmation_by_document=None, metadata={'statement_type': 'descriptive'}),
 CheckedClaim(text='Climate change is a real phenomenon', negation='Climate change is not a real phenomenon.', uid='d86f500a-8ca7-40cd-b1ae-3a8535146f57', n_evidence=None, average_confirmation=None, evidential_uncertainty=None, verbalized_confirmation=None, documents=None, confirmation_by_document=None, metadata={'statement_type': 'descriptive'}),
 CheckedClaim(text='Climate change should be prioritized above other challenges facing humanity.', negation='Climate change should not be prioritized above other challenges facing humanity.', uid='4fda9896-4710-4901-901e-3bdf57721cbf', n_evidence=None, aver